# ATP Tennis Analysis

In [1]:
import pandas as pd
import numpy as np
import re
from glob import glob
from tqdm import tqdm

## Data Loading

In [2]:
# Load all data from 2016-2019
years_to_consider = ["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"]
all_files = ["tennis_atp_data/atp_matches_" + year + ".csv" for year in years_to_consider]

In [3]:
df_all_matches = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

## Data Set Construction

We need 3 different sources of data. For each of the 2 players, we need:
1. All past matches they have played
2. The most recent 3 matches against the same handed opponent on the same court

We also need past H2H data which will be the same for both players and thus does not need to duplicated.

The data should then be stored in data structures with efficient lookup for fast training time.

### Data Cleaning

First, to clean the data:
1. Remove names of tournament and players - these are captured by ids already
2. Remove the match number
3. Remove draw size
4. Remove the IOC (Country of origin)
5. Convert seed to seed score (1/s if s is not NaN, otherwise 0)
6. Convert score to games won for each player
7. Convert entry (take lowercase) into OHE
8. Remove minutes
9. Remove rank
10. Drop best of
11. Drop round
13. Drop tourney level
14. Drop height and age

In [4]:
# 1.
df_all_matches.drop(["tourney_name", "winner_name", "loser_name"], axis = 1, inplace = True)

In [5]:
# 2.
df_all_matches.drop(["match_num"], axis = 1, inplace = True)

In [6]:
# 3.
df_all_matches.drop(["draw_size"], axis = 1, inplace = True)

In [7]:
# 4.
df_all_matches.drop(["winner_ioc", "loser_ioc"], axis = 1, inplace = True)

In [8]:
# 5.

def calculate_seed_score(seed):
    if np.isnan(seed):
        return 0
    return 1/seed

df_all_matches["winner_seed_score"] = df_all_matches["winner_seed"].apply(calculate_seed_score)
df_all_matches["loser_seed_score"] = df_all_matches["loser_seed"].apply(calculate_seed_score)

df_all_matches.drop(["winner_seed", "loser_seed"], axis = 1, inplace = True)

In [9]:
# 6.
def remove_tiebreak_scores(score_str):
    # This removes any parenthetical group like (8), (10), etc.
    cleaned = re.sub(r'\(\d+\)', '', score_str)
    
    # Remove 'RET', 'W/O', 'WO', 'DEF', case-insensitively and any extra whitespace
    cleaned_2 = re.sub(r'\b(RET|W\/O|WO|DEF|Def|Abandoned|and|Played|Default|ABD)\b\.?', '', cleaned, flags=re.IGNORECASE)
    
    # Normalize spaces
    cleaned_3 = ' '.join(cleaned_2.split())
    
    return cleaned_3

def winner_loser_games_won(string_score):
    winner_games = 0
    loser_games = 0
    cleaned_score = remove_tiebreak_scores(string_score)
    sets = cleaned_score.split(" ")
    
    for set in sets:
        if set.startswith("["):
            winner_games += 1
        elif set == "" or set == "Walkover" or set == "":
            pass
        else:
            winner, loser = set.split("-")
            winner_games += float(winner)
            loser_games += float(loser)
    return winner_games, loser_games

df_all_matches[["w_games_won", "l_games_won"]] = df_all_matches["score"].apply(lambda x: pd.Series(winner_loser_games_won(x)))
df_all_matches.drop(["score"], axis = 1, inplace = True)

In [10]:
# 7.

def lower_if_not_nan(entry):
    if type(entry) == float:
        return entry
    return entry.lower()

df_all_matches["winner_entry"] = df_all_matches["winner_entry"].apply(lower_if_not_nan)
df_all_matches["loser_entry"] = df_all_matches["loser_entry"].apply(lower_if_not_nan)
df_all_matches = pd.get_dummies(df_all_matches, columns = ["winner_entry", "loser_entry"], dtype = float)
df_all_matches.drop("loser_entry_s", axis = 1, inplace = True)

In [11]:
# 8.
df_all_matches.drop(["minutes"], axis = 1, inplace = True)

In [12]:
# 9.
df_all_matches.drop(["winner_rank", "loser_rank"], axis = 1, inplace = True)

In [13]:
# 10.
df_all_matches.drop(["best_of"], axis = 1, inplace = True)

In [14]:
# 11.
df_all_matches.drop(["round"], axis = 1, inplace = True)

In [15]:
# 13.
df_all_matches.drop(["tourney_level"], axis = 1, inplace = True)

In [16]:
# 14.
df_all_matches.drop(["winner_age", "winner_ht", "loser_age", "loser_ht"], axis = 1, inplace = True)

In [17]:
# For hand, we will use to create the past X match performance against
# opponents of the same hand then drop that column

In [18]:
df_all_matches.columns

Index(['tourney_id', 'surface', 'tourney_date', 'winner_id', 'winner_hand',
       'loser_id', 'loser_hand', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn',
       'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace',
       'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms',
       'l_bpSaved', 'l_bpFaced', 'winner_rank_points', 'loser_rank_points',
       'winner_seed_score', 'loser_seed_score', 'w_games_won', 'l_games_won',
       'winner_entry_alt', 'winner_entry_ll', 'winner_entry_pr',
       'winner_entry_q', 'winner_entry_se', 'winner_entry_wc',
       'loser_entry_alt', 'loser_entry_ll', 'loser_entry_pr', 'loser_entry_q',
       'loser_entry_se', 'loser_entry_wc'],
      dtype='object')

We can now split the data into train, validation, test - 70%, 15%, 15% chronologically

In [19]:
df_all_matches_train = df_all_matches[:20578]
df_all_matches_validation = df_all_matches[20578:24987]
df_all_matches_test = df_all_matches[24987:]

Fit a normalizer to the training data, then also normalize the validation and test sets with the same normalizer

In [20]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

In [21]:
# Columns to normalize
columns_to_normalize = ['w_ace', 'w_df', 'w_svpt', 'w_1stIn',
       'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace',
       'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms',
       'l_bpSaved', 'l_bpFaced', 'winner_rank_points', 'loser_rank_points',
       'winner_seed_score', 'loser_seed_score', 'w_games_won', 'l_games_won']

# Create the normalizer (fit only on training set)
normalizer = ColumnTransformer(
    transformers=[
        ('norm', MinMaxScaler(), columns_to_normalize)
    ],
    remainder='passthrough'  # Keep non-normalized columns
)
normalizer.fit(df_all_matches_train)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


ColumnTransformer(remainder='passthrough',
                  transformers=[('norm', MinMaxScaler(),
                                 ['w_ace', 'w_df', 'w_svpt', 'w_1stIn',
                                  'w_1stWon', 'w_2ndWon', 'w_SvGms',
                                  'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df',
                                  'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon',
                                  'l_SvGms', 'l_bpSaved', 'l_bpFaced',
                                  'winner_rank_points', 'loser_rank_points',
                                  'winner_seed_score', 'loser_seed_score',
                                  'w_games_won', 'l_games_won'])])

In [22]:
X_train_norm = normalizer.transform(df_all_matches_train)
new_columns = columns_to_normalize + [col for col in df_all_matches_train.columns if col not in columns_to_normalize]
df_all_matches_train = pd.DataFrame(X_train_norm, columns=new_columns)

In [23]:
X_validation_norm = normalizer.transform(df_all_matches_validation)
new_columns = columns_to_normalize + [col for col in df_all_matches_validation.columns if col not in columns_to_normalize]
df_all_matches_validation = pd.DataFrame(X_validation_norm, columns=new_columns)

In [24]:
X_test_norm = normalizer.transform(df_all_matches_test)
new_columns = columns_to_normalize + [col for col in df_all_matches_test.columns if col not in columns_to_normalize]
df_all_matches_test = pd.DataFrame(X_test_norm, columns=new_columns)

In [25]:
df_all_matches_train.columns

Index(['w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms',
       'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stIn',
       'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank_points', 'loser_rank_points', 'winner_seed_score',
       'loser_seed_score', 'w_games_won', 'l_games_won', 'tourney_id',
       'surface', 'tourney_date', 'winner_id', 'winner_hand', 'loser_id',
       'loser_hand', 'winner_entry_alt', 'winner_entry_ll', 'winner_entry_pr',
       'winner_entry_q', 'winner_entry_se', 'winner_entry_wc',
       'loser_entry_alt', 'loser_entry_ll', 'loser_entry_pr', 'loser_entry_q',
       'loser_entry_se', 'loser_entry_wc'],
      dtype='object')

## Input Match

Set up a dataset to represent the data for an upcoming match

In [26]:
df_current_match_train = df_all_matches_train.copy()
df_current_match_valid = df_all_matches_validation.copy()
df_current_match_test = df_all_matches_test.copy()

In [27]:
def preprocess_current_match(df_current_match):
    # Drop all columns relating to the match performance itself
    df_current_match.drop(['w_ace', 'w_df',
           'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved',
           'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon',
           'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced', 'w_games_won', 'l_games_won'], axis = 1, inplace = True)
    # Rename winner and loser to player A and player B and randomly shuffle so that sometimes
    # player A wins and sometime player B wins
    
    # Generate random mask for shuffling
    mask = np.random.rand(len(df_current_match)) < 0.5
    
    # Define base stat fields
    stat_fields = ['id', 'hand', "seed_score", 
                   "entry_alt", "entry_ll",
                   "entry_pr", "entry_q",
                   "entry_se", "entry_wc",
                   "rank_points"
                  ]
    
    for field in stat_fields:
        df_current_match[f'player_A_{field}'] = np.where(mask, df_current_match[f'winner_{field}'], df_current_match[f'loser_{field}'])
        df_current_match[f'player_B_{field}'] = np.where(mask, df_current_match[f'loser_{field}'], df_current_match[f'winner_{field}'])
    
    # Add new column to indicate winner ('A' or 'B')
    df_current_match['winner_label'] = np.where(mask, 'A', 'B')
    
    # Drop original columns
    cols_to_drop = [f'winner_{f}' for f in stat_fields] + [f'loser_{f}' for f in stat_fields]
    df_current_match = df_current_match.drop(columns=cols_to_drop)
    return df_current_match


In [28]:
df_current_match_train = preprocess_current_match(df_current_match_train)
df_current_match_valid = preprocess_current_match(df_current_match_valid)
df_current_match_test = preprocess_current_match(df_current_match_test)

In [29]:
df_current_match_train.columns

Index(['tourney_id', 'surface', 'tourney_date', 'player_A_id', 'player_B_id',
       'player_A_hand', 'player_B_hand', 'player_A_seed_score',
       'player_B_seed_score', 'player_A_entry_alt', 'player_B_entry_alt',
       'player_A_entry_ll', 'player_B_entry_ll', 'player_A_entry_pr',
       'player_B_entry_pr', 'player_A_entry_q', 'player_B_entry_q',
       'player_A_entry_se', 'player_B_entry_se', 'player_A_entry_wc',
       'player_B_entry_wc', 'player_A_rank_points', 'player_B_rank_points',
       'winner_label'],
      dtype='object')

## All Past Performances

In [30]:
def process_past_performance_df(df, prefix, won):
    columns_to_keep = ['ace', 'df', 'svpt', '1stIn', '1stWon',
       '2ndWon', 'SvGms', 'bpSaved', 'bpFaced', "games_won"]
    df_processed = df[[f"{prefix}{col}" for col in columns_to_keep] + ["tourney_date"]].copy()
    df_processed.rename(columns = lambda x: x.replace(prefix, ""), inplace = True)
    df_processed["won"] = float(won)
    return df_processed

In [31]:
def all_past_performances(player_id: int, tourney_date: int, df):
    df_wins = df[(df['winner_id'] == player_id) & (df['tourney_date'] < tourney_date)].copy()
    df_loses = df[(df['loser_id'] == player_id) & (df['tourney_date'] < tourney_date)].copy()

    df_wins_processed = process_past_performance_df(df_wins, "w_", True)
    df_loses_processed = process_past_performance_df(df_loses, "l_", False)

    df_total = pd.concat([df_wins_processed, df_loses_processed], ignore_index = True)

    # Drop rows with incomplete data
    df_total.dropna(inplace = True)
    return df_total

In [32]:
all_past_performances(104745, 20190920, df_all_matches_train)

,ace,df,svpt,1stIn,1stWon,2ndWon,SvGms,bpSaved,bpFaced,games_won,tourney_date,won
0,0.044248,0.0,0.10998,0.099723,0.089041,0.158537,0.1,0.125,0.1,0.130435,20100104,1.0
1,0.026549,0.043478,0.101833,0.088643,0.089041,0.109756,0.088889,0.0,0.0,0.130435,20100104,1.0
2,0.017699,0.0,0.038697,0.033241,0.037671,0.060976,0.044444,0.0,0.0,0.086957,20100104,1.0
3,0.00885,0.0,0.09165,0.077562,0.078767,0.109756,0.088889,0.0,0.033333,0.130435,20100104,1.0
4,0.097345,0.043478,0.211813,0.199446,0.188356,0.219512,0.166667,0.25,0.233333,0.206522,20100118,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
489,0.048544,0.047619,0.216769,0.25,0.172535,0.108911,0.153846,0.32,0.419355,0.131868,20160808,0.0
490,0.009709,0.238095,0.100204,0.10061,0.073944,0.019802,0.087912,0.12,0.258065,0.043956,20160815,0.0
491,0.009709,0.190476,0.306748,0.271341,0.225352,0.29703,0.252747,0.36,0.451613,0.252747,20160829,0.0
492,0.0,0.095238,0.132924,0.146341,0.088028,0.069307,0.098901,0.4,0.483871,0.065934,20161003,0.0


## Past 5 Performance on Same Court and Handed-Opponent

For each player, we want to filter for the past 5 performances against the same handed opponent on the same surface.

In [33]:
def past_5_performance_same_surface(player_id: int, tourney_date: int, surface: str, opponent_hand: str, df):
    df_wins = df[(df['winner_id'] == player_id) & (df['tourney_date'] < tourney_date)].copy()
    df_loses = df[(df['loser_id'] == player_id) & (df['tourney_date'] < tourney_date)].copy()

    df_wins_same_surface_hand = df_wins[(df_wins["surface"] == surface) & (df_wins["loser_hand"] == opponent_hand)].copy()
    df_loses_same_surface_hand = df_loses[(df_loses["surface"] == surface) & (df_loses["winner_hand"] == opponent_hand)].copy()

    df_wins_processed = process_past_performance_df(df_wins_same_surface_hand, "w_", True)
    df_loses_processed = process_past_performance_df(df_loses_same_surface_hand, "l_", False)

    df_total = pd.concat([df_wins_processed, df_loses_processed], ignore_index = True)

    # Drop rows with incomplete data
    df_total.dropna(inplace = True)

    # Sort by tourney_date
    df_total.sort_values(by='tourney_date', ascending=False, inplace=True)
    return df_total[:5]

In [34]:
past_5_performance_same_surface(104745, 20190920, "Clay", "L", df_all_matches_train)

,ace,df,svpt,1stIn,1stWon,2ndWon,SvGms,bpSaved,bpFaced,games_won,tourney_date,won
19,0.0,0.086957,0.14664,0.155125,0.133562,0.109756,0.133333,0.291667,0.3,0.195652,20160523,1.0
18,0.0,0.304348,0.169043,0.182825,0.133562,0.085366,0.122222,0.125,0.233333,0.141304,20150727,1.0
17,0.017699,0.0,0.162933,0.160665,0.126712,0.158537,0.122222,0.291667,0.3,0.163043,20150727,1.0
16,0.0,0.086957,0.077393,0.066482,0.068493,0.109756,0.077778,0.0,0.0,0.130435,20150223,1.0
15,0.017699,0.0,0.087576,0.085873,0.068493,0.097561,0.088889,0.0,0.066667,0.130435,20150216,1.0


## Past H2H Performance

We also want the past H2H performances between the 2 players

In [35]:
def process_h2h(df, winner: str, loser: str):
    performance_columns_to_keep = ['ace', 'df', 'svpt', '1stIn', '1stWon',
       '2ndWon', 'SvGms', 'bpSaved', 'bpFaced', "games_won"]

    df_processed_winner = df[[f"w_{col}" for col in performance_columns_to_keep]].copy()
    df_processed_winner.rename(columns = lambda x: x.replace("w_", winner+"_"), inplace = True)

    df_processed_loser = df[[f"l_{col}" for col in performance_columns_to_keep]].copy()
    df_processed_loser.rename(columns = lambda x: x.replace("l_", loser+"_"), inplace = True)


    df_extra_columns = df[["surface", "tourney_date"]].copy()
    
    df_processed_total = pd.concat([df_processed_winner, df_processed_loser, df_extra_columns], axis = 1)
    df_processed_total["winner"] = float(winner == "A")
    df_processed_total.dropna(inplace = True)
    return df_processed_total

In [36]:
def get_head_to_head(df, player_A_id, player_B_id, tourney_date):
    
    df_player_A_won = df[
        ((df['winner_id'] == player_A_id) & (df['loser_id'] == player_B_id)) &
        (df["tourney_date"] < tourney_date)
        ].copy()
    df_player_A_won_processed = process_h2h(df_player_A_won, "A", "B")
    
    df_player_B_won = df[
        ((df['winner_id'] == player_B_id) & (df['loser_id'] == player_A_id)) &
        (df["tourney_date"] < tourney_date)
        ].copy()
    df_player_B_won_processed = process_h2h(df_player_B_won, "B", "A")

    return pd.concat([df_player_A_won_processed, df_player_B_won_processed])


In [37]:
get_head_to_head(df_all_matches_train, 106233, 133430, 20190920)

,A_ace,A_df,A_svpt,A_1stIn,A_1stWon,A_2ndWon,A_SvGms,A_bpSaved,A_bpFaced,A_games_won,...,B_1stIn,B_1stWon,B_2ndWon,B_SvGms,B_bpSaved,B_bpFaced,B_games_won,surface,tourney_date,winner


## Single data point

For a given row of the Input Match data frame, construct a single data point ready for modelling

In [38]:
df_current_match_train.iloc[0]

tourney_id              2010-339
surface                     Hard
tourney_date            20100103
player_A_id               104053
player_B_id               103429
player_A_hand                  R
player_B_hand                  R
player_A_seed_score          1.0
player_B_seed_score          0.0
player_A_entry_alt           0.0
player_B_entry_alt           0.0
player_A_entry_ll            0.0
player_B_entry_ll            0.0
player_A_entry_pr            0.0
player_B_entry_pr            0.0
player_A_entry_q             0.0
player_B_entry_q             0.0
player_A_entry_se            0.0
player_B_entry_se            0.0
player_A_entry_wc            0.0
player_B_entry_wc            0.0
player_A_rank_points    0.260133
player_B_rank_points    0.035223
winner_label                   A
Name: 0, dtype: object

In [39]:
def single_data_point(df_single_row):
    temp = df_single_row
    player_a_past_performance = all_past_performances(temp["player_A_id"], temp["tourney_date"], df_all_matches_train).drop(["tourney_date"], axis = 1)
    player_a_past_5_performance = past_5_performance_same_surface(
        temp["player_A_id"], temp["tourney_date"], temp["surface"], temp["player_B_hand"], df_all_matches_train).drop(["tourney_date"], axis = 1)
    
    player_b_past_performance = all_past_performances(temp["player_B_id"], temp["tourney_date"], df_all_matches_train).drop(["tourney_date"], axis = 1)
    player_b_past_5_performance = past_5_performance_same_surface(
        temp["player_B_id"], temp["tourney_date"], temp["surface"], temp["player_A_hand"], df_all_matches_train).drop(["tourney_date"], axis = 1)

    past_h2h_data = get_head_to_head(
        df_all_matches_train, temp["player_A_id"], temp["player_B_id"], temp["tourney_date"]
    ).drop(["tourney_date", "surface"], axis = 1)

    temp.drop(["tourney_id", "surface", "tourney_date", "player_A_id", "player_B_id", "player_A_hand", "player_B_hand"], inplace = True)
    temp["winner_label"] = float(temp["winner_label"] == "A")

    return [list(temp), player_a_past_performance, player_b_past_performance, player_a_past_5_performance, player_b_past_5_performance, past_h2h_data]

In [40]:
single_data_point(df_current_match_train.iloc[20000])

/var/folders/_t/7gs3z_zn4c3cqz0nlmg7nk_80000gn/T/ipykernel_78313/2827016905.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.drop(["tourney_id", "surface", "tourney_date", "player_A_id", "player_B_id", "player_A_hand", "player_B_hand"], inplace = True)
/var/folders/_t/7gs3z_zn4c3cqz0nlmg7nk_80000gn/T/ipykernel_78313/2827016905.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["winner_label"] = float(temp["winner_label"] == "A")


[[1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.10820697386276476,
  0.007257065313587822,
  1.0],
           ace        df      svpt     1stIn    1stWon    2ndWon     SvGms  \
 0    0.115044  0.086957  0.179226  0.163435  0.150685  0.170732  0.166667   
 1    0.088496  0.173913  0.244399  0.163435  0.143836  0.426829  0.188889   
 2    0.035398   0.26087  0.183299  0.144044  0.150685  0.219512  0.166667   
 3    0.088496  0.173913   0.14664  0.102493  0.113014  0.231707  0.111111   
 4    0.106195  0.173913  0.169043  0.119114  0.116438  0.268293  0.122222   
 ..        ...       ...       ...       ...       ...       ...       ...   
 373  0.097087  0.047619  0.153374  0.161585  0.137324  0.089109  0.120879   
 374  0.058252  0.047619  0.237219   0.22561  0.158451  0.188119  0.153846   
 375  0.058252  0.190476  0.132924  0.115854  0.091549  0.148515   0.10989   
 376  0.116505  0.095238  0.167689   0.14939  0.123239  0.188119  0

# Model Building

In [41]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Masking, GlobalAveragePooling1D, Concatenate
from tensorflow.keras.losses import BinaryCrossentropy

# === Define input shapes ===
X1 = 11   # Features in time series 1
X2 = 11   # Features in time series 2
dense_input_size = 16

# Inputs (variable-length)
input_ts1 = Input(shape=(None, X1), name='player_a_past_performance')
input_ts2 = Input(shape=(None, X2), name='player_b_past_performance')
input_tab1 = Input(shape=(None, 11), name='player_a_past_5_performance')  # 0–5 rows
input_tab2 = Input(shape=(None, 11), name='player_b_past_5_performance')  # 0–5 rows
input_tab3 = Input(shape=(None, 21), name='h2h_data')  # 0–3 rows
input_dense = Input(shape=(dense_input_size,), name='single_row_16_features')


# === Branch for time series 1 ===
masked_ts1 = Masking()(input_ts1)
lstm_ts1 = LSTM(64)(masked_ts1)

# === Branch for time series 2 ===
masked_ts2 = Masking()(input_ts2)
lstm_ts2 = LSTM(64)(masked_ts2)

# === Branch for tabular data 1 ===
masked_tab1 = Masking()(input_tab1)
dense_tab1 = Dense(32, activation='relu')(masked_tab1)
pooled_tab1 = GlobalAveragePooling1D()(dense_tab1)

# === Branch for tabular data 2 ===
masked_tab2 = Masking()(input_tab2)
dense_tab2 = Dense(32, activation='relu')(masked_tab2)
pooled_tab2 = GlobalAveragePooling1D()(dense_tab2)

# === Branch for tabular data 3 ===
masked_tab3 = Masking()(input_tab3)
dense_tab3 = Dense(32, activation='relu')(masked_tab3)
pooled_tab3 = GlobalAveragePooling1D()(dense_tab3)

dense_branch = Dense(32, activation='relu')(input_dense)

# === Merge all branches ===
merged = Concatenate()([lstm_ts1, lstm_ts2, pooled_tab1, pooled_tab2, pooled_tab3, dense_branch])
dense = Dense(64, activation='relu')(merged)
output = Dense(1, activation='sigmoid')(dense)

model = Model(inputs=[input_ts1, input_ts2, input_tab1, input_tab2, input_tab3, input_dense], outputs=output)
model.compile(optimizer='adam', loss=BinaryCrossentropy(), metrics=['accuracy'])

model.summary()


2025-06-05 18:54:03.080512: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ player_a_past_perf… │ (None, None, 11)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ player_b_past_perf… │ (None, None, 11)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ player_a_past_5_pe… │ (None, None, 11)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ player_b_past_5_pe… │ (None, None, 11)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ h2h_data            │ (None, None, 21)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, None, 11)  │          0 │ player_a_past_pe… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, None, 11)  │          0 │ player_b_past_pe… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_2 (Masking) │ (None, None, 11)  │          0 │ player_a_past_5_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, None, 11)  │          0 │ player_a_past_5_… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_3 (Masking) │ (None, None, 11)  │          0 │ player_b_past_5_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_3         │ (None, None, 11)  │          0 │ player_b_past_5_… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_4 (Masking) │ (None, None, 21)  │          0 │ h2h_data[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_4         │ (None, None, 21)  │          0 │ h2h_data[0][0]    │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking (Masking)   │ (None, None, 11)  │          0 │ player_a_past_pe… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any (Any)           │ (None, None)      │          0 │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_1 (Masking) │ (None, None, 11)  │          0 │ player_b_past_pe… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any_1 (Any)         │ (None, None)      │          0 │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 32)  │        384 │ masking_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any_2 (Any)         │ (None, None)      │          0 │ not_equal_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None, 32)  │        384 │ masking_3[0][0] 

 Total params: 57,441 (224.38 KB)

 Trainable params: 57,441 (224.38 KB)

 Non-trainable params: 0 (0.00 B)

In [42]:
def train_model_samplewise(model, train_data, epochs=1):
    for epoch in range(epochs):
        total_loss = 0.0
        sample_count = 0

        print(f"\n=== Epoch {epoch + 1} ===")
        
        for i, sample in tqdm(train_data.iterrows(), total=len(train_data)):
            try:
                dense_input, ts1, ts2, tab1, tab2, tab3 = single_data_point(sample)

                # Skip incomplete inputs
                if tab1.shape[0] == 0 or tab2.shape[0] == 0 or tab3.shape[0] == 0:
                    continue

                # Ensure all inputs are properly shaped float32 arrays
                ts1 = np.expand_dims(np.array(ts1, dtype=np.float32), axis=0)
                ts2 = np.expand_dims(np.array(ts2, dtype=np.float32), axis=0)
                tab1 = np.expand_dims(np.array(tab1, dtype=np.float32), axis=0)
                tab2 = np.expand_dims(np.array(tab2, dtype=np.float32), axis=0)
                tab3 = np.expand_dims(np.array(tab3, dtype=np.float32), axis=0)

                dense_input_array = np.array(dense_input, dtype=np.float32)
                target_value = float(dense_input_array[-1])  # Ensure it's a scalar float

                dense_input_trimmed = np.expand_dims(dense_input_array[:-1], axis=0)
                target = np.array([[target_value]], dtype=np.float32)

                inputs = [ts1, ts2, tab1, tab2, tab3, dense_input_trimmed]
                for input_arr in [ts1, ts2, tab1, tab2, tab3, dense_input_trimmed]:
                    assert not np.isnan(input_arr).any(), "NaN in input"
                    assert not np.isinf(input_arr).any(), "Inf in input"

                # === Train step ===
                loss = model.train_on_batch(inputs, target)

                # # Log raw loss
                # print(f"Sample {i} — Raw loss return: {loss}")

                # If using metrics, loss is a list
                if isinstance(loss, (list, tuple)):
                    loss_value = loss[1]
                else:
                    loss_value = loss

                total_loss += loss_value
                sample_count += 1

                # Optional: print a prediction every 10 samples
                if i % 10 == 0:
                    pred = model.predict(inputs, verbose=0)
                    print(f"  ➤ Predicted: {pred[0][0]:.4f}, Target: {target_value:.1f}, Loss: {loss_value:.4f}")

            except Exception as e:
                print(f"Error on sample {i}: {e}")
                continue

        # if sample_count == 0:
        #     print("⚠️ No valid samples used this epoch.")
        #     continue

        avg_loss = total_loss / sample_count
        print(f"✅ Epoch {epoch + 1} — Trained on {sample_count} samples — Avg Loss: {avg_loss:.4f}")


In [43]:
train_model_samplewise(model, df_current_match_train[-8000:])


=== Epoch 1 ===


  0%|                                          | 8/8000 [00:07<1:41:49,  1.31it/s]

  0%|                                           | 21/8000 [00:08<14:39,  9.07it/s]

/opt/anaconda3/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['player_a_past_performance', 'player_b_past_performance', 'player_a_past_5_performance', 'player_b_past_5_performance', 'h2h_data', 'single_row_16_features']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*')
  warnings.warn(
  0%|                                           | 23/8000 [00:10<45:07,  2.95it/s]

  ➤ Predicted: 0.4322, Target: 1.0, Loss: 0.5714


...

  ➤ Predicted: 0.3416, Target: 1.0, Loss: 0.6620


100%|██████████████████████████████████████▉| 7983/8000 [1:16:20<00:20,  1.22s/it]

  ➤ Predicted: 0.2690, Target: 1.0, Loss: 0.6613


100%|███████████████████████████████████████| 8000/8000 [1:16:40<00:00,  1.74it/s]

✅ Epoch 1 — Trained on 3837 samples — Avg Loss: 0.6407


In [44]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from tqdm import tqdm

def evaluate_model_samplewise(model, data):
    total_loss = 0.0
    y_true = []
    y_pred = []
    sample_count = 0

    for i, sample in tqdm(data.iterrows(), total=len(data)):
        try:
            dense_input, ts1, ts2, tab1, tab2, tab3 = single_data_point(sample)

            if tab1.shape[0] == 0 or tab2.shape[0] == 0 or tab3.shape[0] == 0:
                continue

            # Format inputs
            ts1 = np.expand_dims(np.array(ts1, dtype=np.float32), axis=0)
            ts2 = np.expand_dims(np.array(ts2, dtype=np.float32), axis=0)
            tab1 = np.expand_dims(np.array(tab1, dtype=np.float32), axis=0)
            tab2 = np.expand_dims(np.array(tab2, dtype=np.float32), axis=0)
            tab3 = np.expand_dims(np.array(tab3, dtype=np.float32), axis=0)

            dense_input_array = np.array(dense_input, dtype=np.float32)
            target_value = float(dense_input_array[-1])  # scalar
            dense_input_trimmed = np.expand_dims(dense_input_array[:-1], axis=0)
            target = np.array([[target_value]], dtype=np.float32)

            inputs = [ts1, ts2, tab1, tab2, tab3, dense_input_trimmed]

            # Prediction
            pred = model.predict(inputs, verbose=0)[0][0]  # scalar

            # Record actual and predicted labels
            y_true.append(int(target_value))
            y_pred.append(int(pred >= 0.5))  # threshold at 0.5

            # Evaluate sample loss
            loss = model.evaluate(inputs, target, verbose=0)
            loss_value = loss[0] if isinstance(loss, (list, tuple)) else loss
            total_loss += loss_value
            sample_count += 1

        except Exception as e:
            print(f"Error on sample {i}: {e}")
            continue

    # Compute metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    avg_loss = total_loss / max(sample_count, 1)

    print("\n📊 Evaluation Metrics:")
    print(f"Samples evaluated: {sample_count}")
    print(f"Average Loss      : {avg_loss:.4f}")
    print(f"Accuracy          : {accuracy:.4f}")
    print(f"Precision         : {precision:.4f}")
    print(f"Recall            : {recall:.4f}")
    print(f"F1 Score          : {f1:.4f}")

    # Optionally return metrics
    return {
        'loss': avg_loss,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [45]:
evaluate_model_samplewise(model, df_current_match_test)

  0%|                                            | 2/4410 [00:00<04:59, 14.70it/s]/opt/anaconda3/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['player_a_past_performance', 'player_b_past_performance', 'player_a_past_5_performance', 'player_b_past_5_performance', 'h2h_data', 'single_row_16_features']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*')
  warnings.warn(
  0%|                                            | 6/4410 [00:01<17:49,  4.12it/s]/opt/anaconda3/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['player_a_past_performance', 'player_b_past_performance', 'player_a_past_5_performance', 'player_b_past_5_performance', 'h2h_data', 'single_row_16_features']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*')
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-pac


📊 Evaluation Metrics:
Samples evaluated: 1197
Average Loss      : nan
Accuracy          : 0.6358
Precision         : 0.6655
Recall            : 0.6190
F1 Score          : 0.6414


{'loss': nan,
 'accuracy': 0.6357560568086884,
 'precision': 0.6655290102389079,
 'recall': 0.6190476190476191,
 'f1': 0.6414473684210527}